# 01 Basic Examples

In [1]:
from abc import ABC, abstractmethod

from pyjudo import ServiceContainer

## Service Resolution
This is the most basic use case.  
We register some services which have interdependencies, register them with the `ServiceContainer`, then retrieve them from the container, allowing it to resolve the dependencies.

In this example, we define two services: `IFoo` and `IBar`, and create an implementation for each: `Foo` and `Bar`.  
The implementation of `Bar` has a dependency on `Foo` - so, when we retrieve a `IBar` from the `ServiceContainer` it is automatically created for us.

In [2]:
# Define some interfaces (abstract classes) and implementations

# Interfaces
class IFoo(ABC):
    @abstractmethod
    def foo(self):
        pass

class IBar(ABC):
    @abstractmethod
    def bar(self):
        pass

# Implementations
class Foo(IFoo):
    def foo(self):
        print("I'm a Foo")

class Bar(IBar):
    def __init__(self, my_foo: IFoo):
        self.my_foo = my_foo
    
    def bar(self):
        print("I'm a Bar")
        self.my_foo.foo()

In [3]:
# Register the services with the container
container = ServiceContainer()

container.register(IFoo, Foo)
container.register(IBar, Bar)

In [4]:
# Retrieve the services from the container
foo = container.get(IFoo)
bar = container.get(IBar)

# Note: You can also retrieve services using the alternate syntax:
# foo = container[IFoo]()
# bar = container[IBar]()

In [5]:
# Use the services
foo.foo()

I'm a Foo


In [6]:
bar.bar()

I'm a Bar
I'm a Foo


Services that include another service in their constructor (i.e. `__init__`), typically referred to as "dependencies", will have them automatically resolved by the `ServiceContainer`.  

In our example above, when we retrieved an `IBar` from the `ServiceContainer` (i.e. `container.get(IBar)`), it recognises it requires an `IFoo` so it created one for us and passed it to the `IBar` constructor.  

## Unresolvable Dependencies
This is a somewhat fancy term for "stuff that the container does't know about".
This could include parameters such as strings (you probably shouldn't be registering strings as a service type...), or flags, etc. that are particular to a service.

In order to pass unresolvable dependencies or parameters to a service constructor, we can pass those arguments to the `ServiceContainer` when we retrieve the service.  

In [7]:
# Define another interface and implementation with an unresolvable dependency

class IBaz(ABC):
    text: str

    @abstractmethod
    def baz(self):
        pass

class Baz(IBaz):
    def __init__(self, text: str):
        self.text = text
    
    def baz(self):
        print(f"I'm a Baz with text: {self.text}")

In [8]:
# Register the service with the container

container.register(IBaz, Baz)

In [9]:
# Try to retrieve the service from the container

from pyjudo.exceptions import ServiceResolutionError

# This will raise a ServicesResolutionError because the container cannot resolve the 
# dependency of 'text' in the Baz constructor (i.e. __init__(self, text: str))
try:
    baz = container.get(IBaz)
except ServiceResolutionError as e:
    print(f"Error: {e!r}")

# We need to give the container the value of 'text' in order to resolve the dependency
baz = container.get(IBaz, text="Hello, World!")

baz.baz()

Error: ServiceResolutionError("Unable to resolve dependency 'text' for 'Baz'")
I'm a Baz with text: Hello, World!


In this example, we have specified an "unresolvable dependency" in `Baz`, i.e. `text`. We can pass this as an argument when retrieving `IBaz` from the `ServiceContainer`.

> **NOTE**  
> Arguments or "overrides" passed to the `ServiceContainer` can only be keyword arguments - it does not support positional. This makes it more explicit about which parameters are being provided.

In [10]:
# If the service has a default value in the constructor, it will be automatically used if an
# overriding argument is not passed to the container.

class BazWithDefault(IBaz):
    def __init__(self, text: str = "Default text"):
        self.text = text
    
    def baz(self):
        print(f"I'm a Baz with text: {self.text}")

# Lets remove the current registration for IBaz and register the new implementation
container.unregister(IBaz)
container.register(IBaz, BazWithDefault)

In [11]:
# Now we can retrieve the service without passing the 'text' argument

baz = container.get(IBaz)

baz.baz()

I'm a Baz with text: Default text
